# Deploy with Docker 

![Vespa logo](https://vespa.ai/assets/vespa-logo-color.png)

This notebook explores how to build, export and deploy an
[application package](https://docs.vespa.ai/en/application-packages.html).

See pyvespa, jupyter notebooks and Docker [requirements](https://pyvespa.readthedocs.io/en/latest/index.html#requirements) to run this notebook.

## Deploy an application package created with pyvespa

This section creates an empty [ApplicationPackage](reference-api.rst#vespa.package.ApplicationPackage) 
instance for deployment testing:

In [ ]:
from vespa.package import ApplicationPackage

app_package = ApplicationPackage(name="sampleapp")

Deploy the `app_package` using Docker without leaving the notebook, by creating an instance of [VespaDocker](reference-api.rst#vespa.deployment.VespaDocker):

In [ ]:
import os
from vespa.deployment import VespaDocker

vespa_docker = VespaDocker()
app = vespa_docker.deploy(application_package=app_package)

Waiting for configuration server, 0/300 seconds...
Waiting for configuration server, 5/300 seconds...
Waiting for application status, 0/300 seconds...
Waiting for application status, 5/300 seconds...
Waiting for application status, 10/300 seconds...
Waiting for application status, 15/300 seconds...
Waiting for application status, 20/300 seconds...
Waiting for application status, 25/300 seconds...
Finished deployment.


`app` now holds a [Vespa](reference-api.rst#vespa.application.Vespa) instance, to be used to interact with the application.
Congratulations, the Vespa application is up and running!

Stop the docker container to clean up:

In [ ]:
vespa_docker.container.stop()
vespa_docker.container.remove()

## Learn Vespa by looking at underlying config files

`pyvespa` provides a convenient API to define Vespa application packages from python.
Going through those files is a nice way to start learning about Vespa syntax.

Use [to_files](reference-api.rst#vespa.package.ApplicationPackage.to_files)
to export the application package package created with `pyvespa` from code to files:

In [ ]:
from pathlib import Path
Path("mydir").mkdir(parents=True, exist_ok=True)
app_package.to_files("mydir")

Inspect the exported files:

In [ ]:
!find mydir

It is also possible to export the application package as a zipped file
using [to_zipfile](reference-api.rst#vespa.package.ApplicationPackage.to_zipfile).
The zipfile can later be deployed with pyvespa or the [Vespa CLI](https://docs.vespa.ai/en/vespa-cli.html):

In [ ]:
app_package.to_zipfile("myzip.zip")

Remove the files after use:

In [ ]:
from shutil import rmtree
rmtree("mydir", ignore_errors=True)
rmtree("myzip.zip", ignore_errors=True)

## Deploy application package from config files

The `pyvespa` API provides a subset of the functionality available in Vespa. The reason is that `pyvespa` is meant to be used as an experimentation tool for Information Retrieval (IR) and not for building production-ready applications. So, the python API expands based on the needs to replicate common use cases that often require IR experimentation.

If the application requires functionality or fine-tuning not available in `pyvespa`, simply build it directly using Vespa configuration files - see the [example](https://pyvespa.readthedocs.io/en/latest/application-packages.html#Example:-configure-max-disk-usage).